# Collider bias {#sec-collider-bias}

**Prerequisiti**

**Concetti e competenze chiave**

**Preparazione del Notebook**

## Introduzione {.unnumbered}

Il concetto del "bias del collider"  è stato approfondito in uno studio recente di  @fitzgerald2024us, che ha reinterpretato i dati di un'analisi precedente condotta da @rains2024us. Lo studio originale confrontava gli stati degli Stati Uniti che avevano implementato mandati di vaccinazione COVID-19 con quelli che, al contrario, avevano vietato tali requisiti. L'obiettivo era valutare come queste politiche avessero influenzato non solo i tassi di vaccinazione COVID-19, ma anche l'adozione di altri vaccini, come quello antinfluenzale.

## Rains e Richards (2024)

Lo studio di @rains2024us sui mandati di vaccinazione COVID-19 negli Stati Uniti ha portato a due conclusioni principali:

1. I mandati statali non hanno avuto un effetto significativo sull’adozione del vaccino COVID-19.
2. Gli stati con mandati di vaccinazione COVID-19 hanno mostrato una minore adozione dei vaccini antinfluenzali e dei richiami del vaccino COVID-19 rispetto agli stati che avevano vietato tali mandati.

## Fitzgerald (2024)

@fitzgerald2024us critica la seconda conclusione dello studio di Rains e Richards, identificando un errore metodologico cruciale. Il problema principale risiede nell'inclusione del tasso di vaccinazione COVID-19 come variabile di controllo nei modelli statistici.

Questo approccio è problematico per tre ragioni:

1. Fattori comuni come l’esitazione vaccinale influenzano sia l'adozione del vaccino COVID-19 che altri vaccini, come quello antinfluenzale.
2. I mandati possono influenzare sia l’adozione dei richiami COVID-19 che dei vaccini antinfluenzali.
3. Controllare i tassi di vaccinazione COVID-19 nei modelli può distorcere i risultati, introducendo un "bias del collider."

::: {#fig-itzgerald-2024}
![](../../figures/fitzgerald_2024.png){width="80%"}

Grafo aciclico diretto che mostra le relazioni tra i mandati di vaccinazione COVID-19, i tassi di vaccinazione COVID-19, l'assunzione di vaccini booster/influenzali COVID-19 e fattori non osservati come l'esitazione nei confronti dei vaccini (Figura tratta da @fitzgerald2024us).
:::

@fitzgerald2024us dimostra che, eliminando questa variabile di controllo errata dai modelli di  @rains2024us, i risultati si invertono. Infatti, gli stati che hanno imposto i mandati di vaccinazione mostrano tassi più alti di adozione sia dei richiami COVID-19 che dei vaccini antinfluenzali.

## Analisi statistica

Di seguito è riportato il codice R utilizzato da @fitzgerald2024us per replicare l'analisi, evidenziando la variazione nei coefficienti relativi alla variabile "Mandate State". Quando il modello include il tasso di vaccinazione COVID-19 come controllo, i coefficienti risultano negativi. Tuttavia, rimuovendo questo controllo, i coefficienti cambiano di segno, diventando positivi, evidenziando l'impatto del collider bias sull'analisi originale.

L'inversione del segno dei coefficienti sottolinea come l'inclusione di una variabile di controllo inappropriata possa portare a conclusioni fuorvianti sull’effetto dei mandati vaccinali sull’adozione dei vaccini. L'analisi di @fitzgerald2024us dimostra l'importanza di considerare attentamente le relazioni causali tra le variabili in studi di epidemiologia e sanità pubblica. La scelta delle variabili di controllo è cruciale per evitare errori metodologici che possono compromettere la validità dei risultati.

In [13]:
library(kableExtra) # V1.3.4
library(here)
library(lme4) # V1.1.34
library(lmerTest) # V3.1.3
library(performance) # V0.10.9
library(gt)

In [4]:
booster_analysis <- read.csv(here("data", "fitzgerald_2024", "booster_analysis.csv"))
flu_analysis_adult <- read.csv(here("data", "fitzgerald_2024", "flu_analysis_adult.csv"))
flu_analysis_children <- read.csv(here("data", "fitzgerald_2024", "flu_analysis_children.csv"))

In [21]:
# Create space for table data
table_data <- as.data.frame(matrix(nrow = 13, ncol = 4))
colnames(table_data) <- NULL

# Input column information
table_data[1, ] <- c("", "COVID-19 Booster Uptake", "Adult Flu Vaccine Uptake", "Child Flu Vaccine Uptake")

###############
### PANEL A ###
###############

# Insert panel information
table_data[2, ] <- c("\\textbf{Panel A: Bad Control Included}", rep("", 3))
# Insert variables and model characteristics
table_data[3, 1] <- "Mandate State"
table_data[4, 1] <- ""
table_data[5, 1] <- "COVID-19"
table_data[6, 1] <- "Vaccination Rate"
table_data[7, 1] <- "$N$"
table_data[8, 1] <- "Conditional/marginal $R^2$"

# COVID-19 booster uptake model
model <- lmer(prop_boosted ~ mandate_type + prop_vacc_center + (1 | state_ab), data = booster_analysis)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[3, 2] <- round(model$coefficients[2, 1], 3)
table_data[4, 2] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[5, 2] <- round(model$coefficients[3, 1], 3)
table_data[6, 2] <- paste0("(", round(model$coefficients[3, 2], 3), ")")
table_data[7, 2] <- length(model$residuals)
table_data[8, 2] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))

# Adult flu vaccine uptake model
model <- lmer(flu_vacc_est ~ mandate_type + cvd_vac_prop_cent + (1 | state_ab), data = flu_analysis_adult)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[3, 3] <- round(model$coefficients[2, 1], 3)
table_data[4, 3] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[5, 3] <- round(model$coefficients[3, 1], 3)
table_data[6, 3] <- paste0("(", round(model$coefficients[3, 2], 3), ")")
table_data[7, 3] <- length(model$residuals)
table_data[8, 3] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))

# Child flu vaccine uptake model
model <- lmer(flu_vacc_est ~ mandate_type + cvd_vac_prop_cent + (1 | state_ab), data = flu_analysis_children)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[3, 4] <- round(model$coefficients[2, 1], 3)
table_data[4, 4] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[5, 4] <- round(model$coefficients[3, 1], 3)
table_data[6, 4] <- paste0("(", round(model$coefficients[3, 2], 3), ")")
table_data[7, 4] <- length(model$residuals)
table_data[8, 4] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))

###############
### PANEL B ###
###############

# Insert panel information
table_data[9, ] <- c("\\textbf{Panel B: Bad Control Removed}", rep("", 3))
# Insert variables and model characteristics
table_data[10, 1] <- "Mandate State"
table_data[11, 1] <- ""
table_data[12, 1] <- "$N$"
table_data[13, 1] <- "Conditional/marginal $R^2$"

# COVID-19 booster uptake model
model <- lmer(prop_boosted ~ mandate_type + (1 | state_ab), data = booster_analysis)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[10, 2] <- round(model$coefficients[2, 1], 3)
table_data[11, 2] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[12, 2] <- length(model$residuals)
table_data[13, 2] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))

# Adult flu vaccine uptake model
model <- lmer(flu_vacc_est ~ mandate_type + (1 | state_ab), data = flu_analysis_adult)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[10, 3] <- round(model$coefficients[2, 1], 3)
table_data[11, 3] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[12, 3] <- length(model$residuals)
table_data[13, 3] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))

# Child flu vaccine uptake model
model <- lmer(flu_vacc_est ~ mandate_type + (1 | state_ab), data = flu_analysis_children)
r2 <- r2(model)
model <- summary(model)
# Insert estimates
table_data[10, 4] <- round(model$coefficients[2, 1], 3)
table_data[11, 4] <- paste0("(", round(model$coefficients[2, 2], 3), ")")
table_data[12, 4] <- length(model$residuals)
table_data[13, 4] <- paste0(round(r2$R2_conditional, 3), "/", round(r2$R2_marginal, 3))


In [26]:
# Produce and save table
table <- kbl(table_data,
    booktabs = TRUE,
    format = "html",
    linesep = c(
        "\\midrule",
        rep("", 4), "\\midrule", rep("", 1), "\\midrule",
        rep("", 2), "\\midrule", rep("", 1), "\\bottomrule"
    ),
    align = c("l", "c", "c", "c", "c", "c", "c", "c"),
    label = "Table_1",
    escape = FALSE
) %>%
    kable_styling(full_width = F) %>%
    footnote(
        general = "Standard errors are reported in parentheses.",
        footnote_as_chunk = TRUE
    )

In [32]:
# Produce and save table
table <- kbl(table_data,
    format = "pipe",
    align = c("l", "c", "c", "c", "c", "c", "c", "c"),
    caption = "Table 1: Effects of Vaccine Mandates on Vaccine Uptake"
)

In [33]:
table
cat("\nNote: Standard errors are reported in parentheses.")



Table: Table 1: Effects of Vaccine Mandates on Vaccine Uptake

|                                       |                         |                          |                          |
|:--------------------------------------|:-----------------------:|:------------------------:|:------------------------:|
|                                       | COVID-19 Booster Uptake | Adult Flu Vaccine Uptake | Child Flu Vaccine Uptake |
|\textbf{Panel A: Bad Control Included} |                         |                          |                          |
|Mandate State                          |         -0.072          |          -0.119          |          -0.179          |
|                                       |         (0.027)         |         (0.021)          |         (0.028)          |
|COVID-19                               |          3.686          |          1.518           |          2.194           |
|Vaccination Rate                       |         (0.056)         |          (0.0


Note: Standard errors are reported in parentheses.

## Considerazioni Conclusive

Il concetto chiave introdotto da @fitzgerald2024us è il **"bias del collider"**. Un "collider" è una variabile influenzata da due o più altre variabili nel modello. Nel caso specifico:

- **I tassi di vaccinazione COVID-19** sono il collider, poiché sono influenzati sia dai mandati vaccinali che da altri fattori, come l’esitazione vaccinale.
- Controllando i tassi di vaccinazione COVID-19 come se fossero una variabile indipendente, si blocca il percorso causale tra i mandati vaccinali e altre variabili, come l’adozione dei richiami o dei vaccini antinfluenzali.
  
Questo porta a una stima distorta dell'effetto dei mandati sui comportamenti vaccinali successivi.

In conclusione, il bias del collider è un errore comune nelle analisi statistiche che può condurre a risultati fuorvianti se non viene identificato e gestito correttamente. Nel caso specifico dei mandati vaccinali, controllare i tassi di vaccinazione COVID-19 come variabile di controllo blocca una parte della relazione causale tra mandati e comportamenti successivi, come l’adozione di richiami o di vaccini antinfluenzali. Questo errore metodologico può portare a conclusioni errate e fuorvianti. Lo stesso principio si applica a studi in altri ambiti, come la psicologia, dove il controllo di variabili che fungono da collider può mascherare relazioni causali importanti e portare a conclusioni errate. È essenziale, quindi, prestare molta attenzione alla scelta delle variabili di controllo per garantire la validità dei risultati negli studi osservazionali.

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [24]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Sat Jul 27 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

networkx   : 3.3
pandas     : 2.2.2
bambi      : 0.14.0
seaborn    : 0.13.2
graphviz   : 0.20.3
numpy      : 1.26.4
arviz      : 0.18.0
matplotlib : 3.9.1
statsmodels: 0.14.2

Watermark: 2.4.3

